In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import rho_plus as rp

is_dark = False
theme, cs = rp.mpl_setup(is_dark)

In [2]:
df = pd.read_pickle('~/cdv/precomputed/mptrj/raw/batch_0000.pkl')
df

,structure,uncorrected_total_energy,corrected_total_energy,energy_per_atom,ef_per_atom,e_per_atom_relaxed,ef_per_atom_relaxed,force,stress,magmom,bandgap,mp_id,calc,step
0,"[[-0.82086724 2.36636286 1.57933414] Li, [-2...",-200.957286,-221.957286,-6.936165,-2.147829,-6.938357,-6.938357,"[[3.656e-05, 0.00123128, -0.0006509], [-6.883e...","[[5.39395745, 0.16978013, 0.16669083], [0.1688...","[0.006, 0.0, -0.003, 0.007, 0.007, -0.001, -0....",0.4372,mp-1290794,0,0
1,"[[-0.8209851 2.36693801 1.57963684] Li, [-2...",-200.956055,-221.956055,-6.936127,-2.147791,-6.938357,-6.938357,"[[0.00268167, 0.00160534, -0.00136527], [0.002...","[[0.22357583, 0.14629669, 0.21247205], [0.1453...",None,NaN,mp-1290794,1,60
2,"[[-0.82097617 2.36710947 1.57967763] Li, [-2...",-200.953908,-221.953908,-6.936060,-2.147724,-6.938357,-6.938357,"[[-0.00018764, 0.00363509, -0.0039542], [0.001...","[[2.27939972, -0.74664765, -0.01904857], [-0.7...",None,NaN,mp-1290794,1,57
3,"[[-0.81565696 2.36922679 1.58542791] Li, [-2...",-200.944425,-221.944425,-6.935763,-2.147427,-6.938357,-6.938357,"[[0.00135505, 0.01206887, -0.00843028], [0.006...","[[4.6597294, -2.18239327, -0.88434682], [-2.18...",None,NaN,mp-1290794,1,47
4,"[[-0.82425815 2.36574523 1.57600746] Li, [-2...",-200.953887,-221.953887,-6.936059,-2.147723,-6.938357,-6.938357,"[[-0.00452961, 0.00255935, -0.00467121], [-0.0...","[[-3.32381681, 2.68401131, 3.19927815], [2.683...",None,NaN,mp-1290794,1,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28071,"[[-1.81414077 -2.71323503 -3.00281063] Hf, [-4...",-54.138093,-57.156093,-7.144512,-1.552878,-7.165131,-7.165131,"[[-0.10479382, -0.0, -0.27046508], [0.10479382...","[[3.33718743, 0.0, 5.63143029], [0.0, -3.88497...",None,NaN,mp-9922,1,9
28072,"[[-1.82863848 -2.69991116 -2.94116506] Hf, [-4...",-54.086891,-57.104891,-7.138111,-1.546478,-7.165131,-7.165131,"[[-0.04764615, -0.0, -0.40686534], [0.04764615...","[[10.28647244, 0.0, 4.65821681], [0.0, 15.9395...",None,NaN,mp-9922,1,8
28073,"[[-1.66175428 -2.81756387 -3.06998331] Hf, [-3...",-52.535076,-55.553076,-6.944135,-1.352501,-7.165131,-7.165131,"[[0.26426779, -0.0, 1.39902493], [-0.26426779,...","[[17.34590664, 0.0, 42.02240376], [0.0, -78.09...",None,NaN,mp-9922,1,2
28074,"[[-1.40908815 -2.6491275 -2.8568699 ] Hf, [-3...",-50.725894,-53.743894,-6.717987,-1.126353,-7.165131,-7.165131,"[[-0.7082542, -0.0, -0.33111347], [0.7082542, ...","[[35.44224199, 0.0, 125.71124928], [0.0, 118.5...",None,NaN,mp-9922,1,0


In [3]:
from crystallographic_graph import make_graph
import pickle
from tqdm import tqdm
import torch

device = torch.device('cuda')

b_inds = df.index[:1]

structs = df.loc[b_inds, 'structure']
struct = structs[0]

def knn_fast(struct, k=16):
    structs = [struct]
    num_atoms = torch.tensor([struct.num_sites for struct in structs], dtype=torch.long, device=device)
    pos = torch.cat([torch.tensor(struct.frac_coords, dtype=torch.float32, device=device) for struct in structs])
    cell = torch.stack([torch.tensor(struct.lattice.matrix, dtype=torch.float32, device=device) for struct in structs])
    edges = make_graph(cell, pos, num_atoms, knn=k)
    # ties are included: remove them
    x = torch.diff(edges.src, prepend=torch.tensor([-1], device=device))
    inds = (torch.nonzero(x) + torch.arange(k, device=device)).flatten()
    return {
        'ims': np.array(edges.cell[inds].reshape(struct.num_sites, k, 3).tolist(), dtype=np.int8),
        'ijs': np.array(edges.dst[inds].reshape(struct.num_sites, k).tolist(), dtype=np.uint16)
    }

knns = []
for struct in tqdm(df.structure):
    knns.append(knn_fast(struct))

with open('knns/batch_0000.pkl', 'wb') as out:
    pickle.dump(knns, out)

100%|██████████| 28076/28076 [05:09<00:00, 90.73it/s] 


In [4]:
from pathlib import Path
Path('~/cdv/precomputed/mptrj/raw/batch_0000.pkl').name

'batch_0000.pkl'

In [5]:
from pathlib import Path
for path in sorted(Path('/home/nmiklaucic/cdv/precomputed/mptrj/raw').glob('batch_*.pkl'))[1:]:
    out_fn = Path('knns') / path.name
    if out_fn.exists():
        continue
    df = pd.read_pickle(path)
    knns = []
    for struct in tqdm(df.structure):
        knns.append(knn_fast(struct))

    with open(out_fn, 'wb') as out:
        pickle.dump(knns, out)

100%|██████████| 44328/44328 [04:50<00:00, 152.60it/s]


In [59]:
structs = [struct]
num_atoms = torch.tensor([struct.num_sites for struct in structs], dtype=torch.long, device=device)
pos = torch.cat([torch.tensor(struct.frac_coords, dtype=torch.float32, device=device) for struct in structs])
cell = torch.stack([torch.tensor(struct.lattice.matrix, dtype=torch.float32, device=device) for struct in structs])
edges = make_graph(cell, pos, num_atoms, knn=16)    

In [78]:
x = torch.diff(edges.src, prepend=torch.tensor([-1], device=device))
(torch.nonzero(x) + torch.arange(16, device=device)).flatten()

tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 1

In [57]:
40 * 16

640

In [50]:
1932 / 3

644.0

In [39]:
def knn_graph(struct, r_start=8, k=16):
    """returns (ijs, ims)
    ijs: nodes k
    ims: nodes k 3"""
    if r_start > 100:
        raise ValueError()
    graph_ijs = []
    graph_ims = []
    r = r_start
    for i, nbs in enumerate(struct.get_all_neighbors(r)):
        sites, dists, idxs, ims = zip(*nbs)
        if len(dists) < k:
            # print('Not enough neighbors, using r =', 2 * r_start)
            return knn_graph(struct, r_start=r * 2, k=k)
        
        chosen = np.argsort(dists)[:k]
        if i == 0:
            print(dists[chosen[-1]])

        graph_ijs.append(np.array(idxs)[chosen])
        graph_ims.append(np.array(ims)[chosen])

    graph_ijs = np.stack(graph_ijs).astype(np.uint16)
    graph_ims = np.stack(graph_ims).astype(np.int8)

    return {'ims': np.array(graph_ims), 'ijs': np.array(graph_ijs)}

edges2 = knn_graph(struct)

2.9465403708936604


In [36]:
print(edges.cell[:16], edges.dst[:16])

tensor([[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0, -1],
        [-1,  0,  0],
        [-1,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        [-1,  0, -1],
        [-1, -1,  0],
        [ 0,  0,  0],
        [ 0,  0, -1],
        [ 0,  0,  0],
        [-1,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0, -1,  0]], device='cuda:0') tensor([19, 16, 30, 27, 14, 12, 28, 25, 13, 15,  2,  2,  9,  6,  7,  8],
       device='cuda:0')


In [33]:
print(edges2['ims'][0], edges2['ijs'][0])

[[ 0  0  0]
 [ 0  0  0]
 [-1  0  0]
 [ 0  0 -1]
 [ 0  0  0]
 [-1  0 -1]
 [ 0  0  0]
 [ 0 -1  0]
 [-1  0  0]
 [ 0  0  0]
 [-1  0  0]
 [ 0  0  0]
 [-1  0  0]
 [ 0  0  0]
 [ 0  0  0]
 [-1 -1  0]] [19 16 27 30 28 25  7  8  9  6 14 12  5  5 15 13]


In [41]:
struct.get_distance(0, 13, (-1, -1, 0))

2.9465403708936604

In [42]:
struct.get_distance(0, 8, (0, -1, 0))

2.8550128626141515